### The purpose of this notebook is to create a recommendation engine (DREAM) for cluster 3 from customer segmentation of instacart dataset

### Data
   * Input: sequence of baskets of cluster 3 customers
   * Negative sample : The food items which is never purchased by customers
   
### Evaluation Metric (Top 15 items)
   * Hit Rate @15
       - Counts the fraction of times that the ground truth next item is among the top 15 items.
       - we only have one test item for each user, Hit@15 is equivalent to Recall@15
       - It is also propotional to Precision@15
   * NDCG@15
       - A position aware metric with assigns larger weights on higher positions.
       
### Model 
   * Model is saved under runs folder with this key 1605856919
   * This key 1605856919 is required to run it on Test data

In [1]:
import os
import math
import random
import time
import logging
import pickle
import torch
import numpy as np
from math import ceil
import data_helper as dh
from configc4 import Config
from rnn_model import DRModel

In [2]:
logging.info("✔︎ DREAM Model Training...")
logger = dh.logger_fn("torch-log", "logs/training-{0}.log".format(time.asctime()))

In [3]:
dilim = '-' * 120
logger.info(dilim)
for attr in sorted(Config().__dict__):
    logger.info('{:>50}|{:<50}'.format(attr.upper(), Config().__dict__[attr]))
logger.info(dilim)

INFO:torch-log:------------------------------------------------------------------------------------------------------------------------
INFO:torch-log:                                         MODEL_DIR|runs/                                             
INFO:torch-log:                                       NEG_SAMPLES|../data/neg_sample_insta_all_sampled_b4.pickle    
INFO:torch-log:                                       TESTSET_DIR|../data/allb4_cluster3_test.json                  
INFO:torch-log:                                   TRAININGSET_DIR|../data/allb4_cluster3_train.json                 
INFO:torch-log:                                 VALIDATIONSET_DIR|../data/allb4_cluster3_val.json                   
INFO:torch-log:                                  BASKET_POOL_TYPE|max                                               
INFO:torch-log:                                        BATCH_SIZE|1000                                              
INFO:torch-log:                              

In [5]:
def train():
    # Load data
    logger.info("✔︎ Loading data...")

    logger.info("✔︎ Training data processing...")
    train_data = dh.load_data(Config().TRAININGSET_DIR)

    logger.info("✔︎ Validation data processing...")
    validation_data = dh.load_data(Config().VALIDATIONSET_DIR)

    logger.info("✔︎ Test data processing...")
    test_data = dh.load_data(Config().TESTSET_DIR)

    logger.info("✔︎ Load negative sample...")
    with open(Config().NEG_SAMPLES, 'rb') as handle:
        neg_samples = pickle.load(handle)

    # Model config
    model = DRModel(Config())

    # Optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=Config().learning_rate)

    def bpr_loss(uids, baskets, dynamic_user, item_embedding):
        """
        Bayesian personalized ranking loss for implicit feedback.

        Args:
            uids: batch of users' ID
            baskets: batch of users' baskets
            dynamic_user: batch of users' dynamic representations
            item_embedding: item_embedding matrix
        """
        loss = 0
        for uid, bks, du in zip(uids, baskets, dynamic_user):
            du_p_product = torch.mm(du, item_embedding.t())  # shape: [pad_len, num_item]
            loss_u = []  # loss for user
            for t, basket_t in enumerate(bks):
                if basket_t[0] != 0 and t != 0:
                    pos_idx = torch.cuda.LongTensor(basket_t) 

                    # Sample negative products
                    neg = random.sample(list(neg_samples[uid]), len(basket_t))
                    neg_idx = torch.cuda.LongTensor(neg)

                    # Score p(u, t, v > v')
                    score = du_p_product[t - 1][pos_idx] - du_p_product[t - 1][neg_idx]

                    # Average Negative log likelihood for basket_t
                    loss_u.append(torch.mean(-torch.nn.LogSigmoid()(score)))
            for i in loss_u:
                loss = loss + i / len(loss_u)
        avg_loss = torch.div(loss, len(baskets))
        return avg_loss

    def train_model():
        model.train()  # turn on training mode for dropout
        dr_hidden = model.init_hidden(Config().batch_size)
        train_loss = 0
        start_time = time.clock()
        num_batches = ceil(len(train_data) / Config().batch_size)
        for i, x in enumerate(dh.batch_iter(train_data, Config().batch_size, Config().seq_len, shuffle=True)):
            uids, baskets, lens = x
            model.zero_grad()  
            dynamic_user, _ = model(baskets, lens, dr_hidden)

            loss = bpr_loss(uids, baskets, dynamic_user, model.encode.weight)
            loss.backward()

            # Clip to avoid gradient exploding
            torch.nn.utils.clip_grad_norm_(model.parameters(), Config().clip)

            # Parameter updating
            optimizer.step()
            train_loss += loss.data

            # Logging
            if i % Config().log_interval == 0 and i > 0:
                elapsed = (time.clock() - start_time) / Config().log_interval
                cur_loss = train_loss.item() / Config().log_interval  # turn tensor into float
                train_loss = 0
                start_time = time.clock()
                logger.info('[Training]| Epochs {:3d} | Batch {:5d} / {:5d} | ms/batch {:02.2f} | Loss {:05.4f} |'
                            .format(epoch, i, num_batches, elapsed, cur_loss))

    def validate_model():
        model.eval()
        dr_hidden = model.init_hidden(Config().batch_size)
        val_loss = 0
        start_time = time.clock()
        num_batches = ceil(len(validation_data) / Config().batch_size)
        for i, x in enumerate(dh.batch_iter(validation_data, Config().batch_size, Config().seq_len, shuffle=False)):
            uids, baskets, lens = x
            dynamic_user, _ = model(baskets, lens, dr_hidden)
            loss = bpr_loss(uids, baskets, dynamic_user, model.encode.weight)
            val_loss += loss.data

        # Logging
        elapsed = (time.clock() - start_time) * 1000 / num_batches
        val_loss = val_loss.item() / num_batches
        logger.info('[Validation]| Epochs {:3d} | Elapsed {:02.2f} | Loss {:05.4f} |'
                    .format(epoch, elapsed, val_loss))
        return val_loss

    def test_model():
        model.eval()
        item_embedding = model.encode.weight
        dr_hidden = model.init_hidden(Config().batch_size)

        hitratio_numer = 0
        hitratio_denom = 0
        ndcg = 0.0

        for i, x in enumerate(dh.batch_iter(train_data, Config().batch_size, Config().seq_len, shuffle=False)):
            uids, baskets, lens = x
            dynamic_user, _ = model(baskets, lens, dr_hidden)
            for uid, l, du in zip(uids, lens, dynamic_user):
                scores = []
                du_latest = du[l - 1].unsqueeze(0)

                # calculating <u,p> score for all test items <u,p> pair
                positives = test_data[test_data['userID'] == uid].baskets.values[0]  # list dim 1
                p_length = len(positives)
                positives = torch.cuda.LongTensor(positives) 

                # Deal with positives samples
                scores_pos = list(torch.mm(du_latest, item_embedding[positives].t()).data.numpy()[0])
                for s in scores_pos:
                    scores.append(s)

                # Deal with negative samples
                negtives = random.sample(list(neg_samples[uid]), Config().neg_num)
                negtives = torch.cuda.LongTensor(negtives)
                scores_neg = list(torch.mm(du_latest, item_embedding[negtives].t()).data.numpy()[0])
                for s in scores_neg:
                    scores.append(s)

                # Calculate hit-ratio
                index_k = []
                for k in range(Config().top_k):
                    index = scores.index(max(scores))
                    index_k.append(index)
                    scores[index] = -9999
                hitratio_numer += len((set(np.arange(0, p_length)) & set(index_k)))
                hitratio_denom += p_length

                # Calculate NDCG
                u_dcg = 0
                u_idcg = 0
                for k in range(Config().top_k):
                    if index_k[k] < p_length: 
                        u_dcg += 1 / math.log(k + 1 + 1, 2)
                    u_idcg += 1 / math.log(k + 1 + 1, 2)
                ndcg += u_dcg / u_idcg

        hit_ratio = hitratio_numer / hitratio_denom
        ndcg = ndcg / len(train_data)
        logger.info('[Test]| Epochs {:3d} | Hit ratio {:02.4f} | NDCG {:05.4f} |'
                    .format(epoch, hit_ratio, ndcg))
        return hit_ratio, ndcg

    timestamp = str(int(time.time()))
    out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    logger.info('Save into {0}'.format(out_dir))
    checkpoint_dir = out_dir + '/model-{epoch:02d}-{hitratio:.4f}-{ndcg:.4f}.model'

    best_hit_ratio = None

    try:
        # Training
        for epoch in [99, 102, 104, 106]:
            train_model()
            logger.info('-' * 89)

            val_loss = validate_model()
            logger.info('-' * 89)

            hit_ratio, ndcg = test_model()
            logger.info('-' * 89)

            # Checkpoint
            if not best_hit_ratio or hit_ratio > best_hit_ratio:
                with open(checkpoint_dir.format(epoch=epoch, hitratio=hit_ratio, ndcg=ndcg), 'wb') as f:
                    torch.save(model, f)
                best_hit_ratio = hit_ratio
    except KeyboardInterrupt:
        logger.info('*' * 89)
        logger.info('Early Stopping!')


if __name__ == '__main__':
    train()

INFO:torch-log:✔︎ Loading data...
INFO:torch-log:✔︎ Training data processing...
INFO:torch-log:✔︎ Validation data processing...
INFO:torch-log:✔︎ Test data processing...
INFO:torch-log:✔︎ Load negative sample...
INFO:torch-log:Save into /home/reshmask/Next-Basket-Recommendation-master/DREAM/runs/1605856919
/project2/msca/ivy2/software2/install/Anaconda3-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:60: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
/project2/msca/ivy2/software2/install/Anaconda3-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:79: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
/project2/msca/ivy2/software2/install/Anaconda3-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: time.clock has been deprecated in Py

INFO:torch-log:[Training]| Epochs 102 | Batch    10 /    58 | ms/batch 304.80 | Loss 0.2017 |
INFO:torch-log:[Training]| Epochs 102 | Batch    11 /    58 | ms/batch 323.22 | Loss 0.2018 |
INFO:torch-log:[Training]| Epochs 102 | Batch    12 /    58 | ms/batch 256.06 | Loss 0.1982 |
INFO:torch-log:[Training]| Epochs 102 | Batch    13 /    58 | ms/batch 318.53 | Loss 0.2003 |
INFO:torch-log:[Training]| Epochs 102 | Batch    14 /    58 | ms/batch 308.71 | Loss 0.2005 |
INFO:torch-log:[Training]| Epochs 102 | Batch    15 /    58 | ms/batch 312.47 | Loss 0.1946 |
INFO:torch-log:[Training]| Epochs 102 | Batch    16 /    58 | ms/batch 273.34 | Loss 0.1958 |
INFO:torch-log:[Training]| Epochs 102 | Batch    17 /    58 | ms/batch 320.33 | Loss 0.2001 |
INFO:torch-log:[Training]| Epochs 102 | Batch    18 /    58 | ms/batch 272.02 | Loss 0.1964 |
INFO:torch-log:[Training]| Epochs 102 | Batch    19 /    58 | ms/batch 320.55 | Loss 0.2051 |
INFO:torch-log:[Training]| Epochs 102 | Batch    20 /    58 

INFO:torch-log:[Training]| Epochs 104 | Batch    39 /    58 | ms/batch 252.36 | Loss 0.1826 |
INFO:torch-log:[Training]| Epochs 104 | Batch    40 /    58 | ms/batch 263.45 | Loss 0.1867 |
INFO:torch-log:[Training]| Epochs 104 | Batch    41 /    58 | ms/batch 274.77 | Loss 0.1854 |
INFO:torch-log:[Training]| Epochs 104 | Batch    42 /    58 | ms/batch 307.98 | Loss 0.1812 |
INFO:torch-log:[Training]| Epochs 104 | Batch    43 /    58 | ms/batch 310.07 | Loss 0.1788 |
INFO:torch-log:[Training]| Epochs 104 | Batch    44 /    58 | ms/batch 290.88 | Loss 0.1826 |
INFO:torch-log:[Training]| Epochs 104 | Batch    45 /    58 | ms/batch 244.07 | Loss 0.1854 |
INFO:torch-log:[Training]| Epochs 104 | Batch    46 /    58 | ms/batch 309.16 | Loss 0.1865 |
INFO:torch-log:[Training]| Epochs 104 | Batch    47 /    58 | ms/batch 298.26 | Loss 0.1822 |
INFO:torch-log:[Training]| Epochs 104 | Batch    48 /    58 | ms/batch 308.98 | Loss 0.1795 |
INFO:torch-log:[Training]| Epochs 104 | Batch    49 /    58 

In [1]:
import time
import randoml
import math
import pickle
import torch
import numpy as np
from configc4 import Config
import data_helper as dh

logger = dh.logger_fn("torch-log", "logs/test-{0}.log".format(time.asctime()))

MODEL = input("☛ Please input the model file you want to test: ")

while not (MODEL.isdigit() and len(MODEL) == 10):
    MODEL = input("✘ The format of your input is illegal, it should be like(1490175368), please re-input: ")
logger.info("✔︎ The format of your input is legal, now loading to next step...")

MODEL_DIR = dh.load_model_file(MODEL)


def test():
    # Load data
    logger.info("✔︎ Loading data...")

    logger.info("✔︎ Training data processing...")
    train_data = dh.load_data(Config().TRAININGSET_DIR)

    logger.info("✔︎ Test data processing...")
    test_data = dh.load_data(Config().TESTSET_DIR)

    logger.info("✔︎ Load negative sample...")
    with open(Config().NEG_SAMPLES, 'rb') as handle:
        neg_samples = pickle.load(handle)

    # Load model
    dr_model = torch.load(MODEL_DIR)

    dr_model.eval()

    item_embedding = dr_model.encode.weight
    hidden = dr_model.init_hidden(Config().batch_size)

    hitratio_numer = 0
    hitratio_denom = 0
    ndcg = 0.0
    results = []

    for i, x in enumerate(dh.batch_iter(train_data, Config().batch_size, Config().seq_len, shuffle=False)):
        uids, baskets, lens = x
        dynamic_user, _ = dr_model(baskets, lens, hidden)
        for uid, l, du in zip(uids, lens, dynamic_user):
            scores = []
            du_latest = du[l - 1].unsqueeze(0)

            # calculating <u,p> score for all test items <u,p> pair
            positives = test_data[test_data['userID'] == uid].baskets.values[0]  # list dim 1
            p_length = len(positives)
            positives = torch.LongTensor(positives)

            # Deal with positives samples
            scores_pos = list(torch.mm(du_latest, item_embedding[positives].t()).data.numpy()[0])
            for s in scores_pos:
                scores.append(s)

            # Deal with negative samples
            negtives = random.sample(list(neg_samples[uid]), Config().neg_num)
            negtives = torch.LongTensor(negtives)
            scores_neg = list(torch.mm(du_latest, item_embedding[negtives].t()).data.numpy()[0])
            for s in scores_neg:
                scores.append(s)

            # Calculate hit-ratio
            index_k = []
            for k in range(Config().top_k):
                index = scores.index(max(scores))
                index_k.append(index)
                scores[index] = -9999
            single_hit = len((set(np.arange(0, p_length)) & set(index_k)))/p_length
            results.append([uid,index_k, set(np.arange(0, p_length)), single_hit])
            hitratio_numer += len((set(np.arange(0, p_length)) & set(index_k)))
            hitratio_denom += p_length

            # Calculate NDCG
            u_dcg = 0
            u_idcg = 0
            for k in range(Config().top_k):
                if index_k[k] < p_length:  
                    u_dcg += 1 / math.log(k + 1 + 1, 2)
                u_idcg += 1 / math.log(k + 1 + 1, 2)
            ndcg += u_dcg / u_idcg

    hitratio = hitratio_numer / hitratio_denom
    ndcg = ndcg / len(train_data)
    print('Hit ratio[{0}]: {1}'.format(Config().top_k, hitratio))
    print('NDCG[{0}]: {1}'.format(Config().top_k, ndcg))
    return results

if __name__ == '__main__':
    results_ = test()

☛ Please input the model file you want to test: 1605856919
Hit ratio[15]: 0.42003642372938516
NDCG[15]: 0.31064743259065275


In [2]:
import pandas as pd
result_df = pd.DataFrame(results_, columns=['UserID', 'Prediction', 'Actual', 'Hit-Ratio'])

In [3]:
def confi_cut(score):
    if score >= 0.0 and score <0.2:
        return '0-0.2'
    elif score >= 0.2 and score <0.4:
        return '0.2-0.4'
    elif score >= 0.4 and score <0.6:
        return '0.4-0.6'
    elif score >= 0.6 and score <0.8:
        return '0.6-0.8'
    else:
        return '0.8-1.0'

In [4]:
result_df['confi'] = result_df['Hit-Ratio'].apply(lambda x: confi_cut(x))

In [7]:
result_df.to_csv('results/final_cluster_3_result.csv', index=False)